In [1]:
import json
from sklearn.cluster import OPTICS
from sklearn.mixture import GaussianMixture
import numpy as np
import os

In [2]:
path_to_data = os.path.join(
    '/mnt',
    'chromeos',
    'removable',
    'jack_sdcard',
    'smite_data',
    'conquest_match_data',
    '1649.json'
)


In [3]:
with open(path_to_data, 'r') as infile:
    raw_data = json.loads(infile.readline())

In [4]:
npdata = np.array([np.divide(np.array([x['assists'], x['damage_mitigated'], x['damage_player'], x['damage_taken'], x['deaths'], x['healing'], x['healing_player_self'], x['kills_player'], x['structure_damage']]), x['match_time_minutes']) for x in raw_data])
npdata_normalized = npdata / npdata.max(axis=0)
npdata_winlabel = np.array([1 if x['win_status'] == 'Winner' else 0 for x in raw_data])

In [5]:
npdata_normalized.shape

(3227, 9)

In [89]:
clustering = OPTICS(
    min_cluster_size=0.1, # minimum cluster size is 1/10th of total
    xi=0.001, # the slope required to form a cluster is 20%
    max_eps=0.13
).fit(npdata_normalized)

In [90]:
unique, counts = np.unique(clustering.labels_, return_counts=True)
dict(zip(unique, counts))


{-1: 2893, 0: 334}

In [76]:
print('20:', np.percentile(clustering.reachability_, 20))
print('50:', np.percentile(clustering.reachability_, 50))
print('80:', np.percentile(clustering.reachability_, 80))

20: 0.08827687060204366
50: 0.10800313302091903
80: 0.13837724755307185


In [9]:
model = GaussianMixture(n_components=3)
model.fit(npdata_normalized[:3000], npdata_winlabel[:3000])
model.score(npdata_normalized[3000:], npdata_winlabel[:3000])

14.78033316269991